## Byg en logistisk regressionsmodel - Lektion 4

![Infografik om logistisk vs. lineær regression](../../../../../../2-Regression/4-Logistic/images/linear-vs-logistic.png)

#### **[Quiz før lektionen](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/15/)**

#### Introduktion

I denne sidste lektion om regression, en af de grundlæggende *klassiske* ML-teknikker, vil vi tage et kig på logistisk regression. Du kan bruge denne teknik til at finde mønstre og forudsige binære kategorier. Er dette slik chokolade eller ej? Er denne sygdom smitsom eller ej? Vil denne kunde vælge dette produkt eller ej?

I denne lektion vil du lære:

-   Teknikker til logistisk regression

✅ Uddyb din forståelse af at arbejde med denne type regression i dette [Learn-modul](https://learn.microsoft.com/training/modules/introduction-classification-models/?WT.mc_id=academic-77952-leestott)

## Forudsætning

Efter at have arbejdet med græskardataene, er vi nu tilstrækkeligt bekendte med dem til at indse, at der er én binær kategori, vi kan arbejde med: `Color`.

Lad os bygge en logistisk regressionsmodel for at forudsige, givet nogle variabler, *hvilken farve et givet græskar sandsynligvis har* (orange 🎃 eller hvid 👻).

> Hvorfor taler vi om binær klassifikation i en lektion, der handler om regression? Kun for sproglig bekvemmelighed, da logistisk regression [faktisk er en klassifikationsmetode](https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression), omend en lineær-baseret en. Lær om andre måder at klassificere data på i den næste lektion.

Til denne lektion skal vi bruge følgende pakker:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) er en [samling af R-pakker](https://www.tidyverse.org/packages), der er designet til at gøre datavidenskab hurtigere, nemmere og sjovere!

-   `tidymodels`: [tidymodels](https://www.tidymodels.org/) er en [samling af pakker](https://www.tidymodels.org/packages/) til modellering og maskinlæring.

-   `janitor`: [janitor-pakken](https://github.com/sfirke/janitor) tilbyder simple værktøjer til at undersøge og rense beskidte data.

-   `ggbeeswarm`: [ggbeeswarm-pakken](https://github.com/eclarke/ggbeeswarm) giver metoder til at lave beeswarm-stil plots ved hjælp af ggplot2.

Du kan installere dem som:

`install.packages(c("tidyverse", "tidymodels", "janitor", "ggbeeswarm"))`

Alternativt kan scriptet nedenfor kontrollere, om du har de nødvendige pakker til at fuldføre dette modul, og installere dem for dig, hvis de mangler.


In [ ]:
suppressWarnings(if (!require("pacman"))install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, janitor, ggbeeswarm)


## **Definer spørgsmålet**

For vores formål vil vi udtrykke dette som en binær: 'Hvid' eller 'Ikke Hvid'. Der er også en 'stribet' kategori i vores datasæt, men der er få eksempler på den, så vi vil ikke bruge den. Den forsvinder alligevel, når vi fjerner null-værdier fra datasættet.

> 🎃 Sjovt faktum, vi kalder nogle gange hvide græskar for 'spøgelsesgræskar'. De er ikke særlig nemme at skære i, så de er ikke lige så populære som de orange, men de ser seje ud! Så vi kunne også omformulere vores spørgsmål som: 'Spøgelse' eller 'Ikke Spøgelse'. 👻

## **Om logistisk regression**

Logistisk regression adskiller sig fra lineær regression, som du tidligere har lært om, på nogle vigtige måder.

#### **Binær klassifikation**

Logistisk regression tilbyder ikke de samme funktioner som lineær regression. Førstnævnte giver en forudsigelse om en `binær kategori` ("orange eller ikke orange"), mens sidstnævnte er i stand til at forudsige `kontinuerlige værdier`, for eksempel givet oprindelsen af et græskar og tidspunktet for høsten, *hvor meget prisen vil stige*.

![Infografik af Dasani Madipalli](../../../../../../2-Regression/4-Logistic/images/pumpkin-classifier.png)

### Andre klassifikationer

Der findes andre typer af logistisk regression, herunder multinomial og ordinal:

- **Multinomial**, som involverer mere end én kategori - "Orange, Hvid og Stribet".

- **Ordinal**, som involverer ordnede kategorier, nyttigt hvis vi ønskede at ordne vores resultater logisk, som vores græskar, der er ordnet efter et begrænset antal størrelser (mini,sm,med,lg,xl,xxl).

![Multinomial vs ordinal regression](../../../../../../2-Regression/4-Logistic/images/multinomial-vs-ordinal.png)

#### **Variabler BEHØVER IKKE at korrelere**

Kan du huske, hvordan lineær regression fungerede bedre med mere korrelerede variabler? Logistisk regression er det modsatte - variablerne behøver ikke at være i overensstemmelse. Det fungerer for disse data, som har forholdsvis svage korrelationer.

#### **Du har brug for en masse rene data**

Logistisk regression vil give mere præcise resultater, hvis du bruger flere data; vores lille datasæt er ikke optimalt til denne opgave, så husk det.

✅ Tænk over, hvilke typer data der egner sig godt til logistisk regression

## Øvelse - ryd op i dataene

Først skal du rydde lidt op i dataene, fjerne null-værdier og kun vælge nogle af kolonnerne:

1. Tilføj følgende kode:


In [ ]:
# Load the core tidyverse packages
library(tidyverse)

# Import the data and clean column names
pumpkins <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/2-Regression/data/US-pumpkins.csv") %>% 
  clean_names()

# Select desired columns
pumpkins_select <- pumpkins %>% 
  select(c(city_name, package, variety, origin, item_size, color)) 

# Drop rows containing missing values and encode color as factor (category)
pumpkins_select <- pumpkins_select %>% 
  drop_na() %>% 
  mutate(color = factor(color))

# View the first few rows
pumpkins_select %>% 
  slice_head(n = 5)


Du kan altid tage et kig på din nye dataframe ved at bruge [*glimpse()*](https://pillar.r-lib.org/reference/glimpse.html)-funktionen som vist nedenfor:


In [ ]:
pumpkins_select %>% 
  glimpse()


Lad os bekræfte, at vi faktisk vil arbejde med et binært klassifikationsproblem:


In [ ]:
# Subset distinct observations in outcome column
pumpkins_select %>% 
  distinct(color)


### Visualisering - kategorisk plot
Du har nu igen indlæst græskardataene og renset dem, så du har et datasæt med nogle få variabler, herunder Farve. Lad os visualisere dataframen i notebooken ved hjælp af ggplot-biblioteket.

Ggplot-biblioteket tilbyder nogle smarte måder at visualisere dine data på. For eksempel kan du sammenligne fordelingen af data for hver Sort og Farve i et kategorisk plot.

1. Opret et sådant plot ved at bruge geombar-funktionen, anvend vores græskardata, og angiv en farvemapping for hver græskarkategori (orange eller hvid):


In [ ]:
# Specify colors for each value of the hue variable
palette <- c(ORANGE = "orange", WHITE = "wheat")

# Create the bar plot
ggplot(pumpkins_select, aes(y = variety, fill = color)) +
  geom_bar(position = "dodge") +
  scale_fill_manual(values = palette) +
  labs(y = "Variety", fill = "Color") +
  theme_minimal()

Ved at observere dataene kan du se, hvordan Farve-dataene relaterer sig til Sort.

✅ Givet dette kategoriske plot, hvilke interessante undersøgelser kan du forestille dig?


### Databehandling: funktionkodning

Vores græskar-datasæt indeholder strengværdier for alle dets kolonner. At arbejde med kategoriske data er intuitivt for mennesker, men ikke for maskiner. Maskinlæringsalgoritmer fungerer godt med tal. Derfor er kodning et meget vigtigt trin i databehandlingsfasen, da det gør det muligt for os at omdanne kategoriske data til numeriske data uden at miste nogen information. God kodning fører til opbygning af en god model.

Der er to hovedtyper af kodere til funktionkodning:

1. Ordinal koder: Den passer godt til ordinale variabler, som er kategoriske variabler, hvor deres data følger en logisk rækkefølge, som kolonnen `item_size` i vores datasæt. Den skaber en mapping, så hver kategori repræsenteres af et tal, som er rækkefølgen af kategorien i kolonnen.

2. Kategorisk koder: Den passer godt til nominelle variabler, som er kategoriske variabler, hvor deres data ikke følger en logisk rækkefølge, som alle funktionerne bortset fra `item_size` i vores datasæt. Det er en one-hot kodning, hvilket betyder, at hver kategori repræsenteres af en binær kolonne: den kodede variabel er lig med 1, hvis græskarret tilhører den pågældende sort, og 0 ellers.

Tidymodels tilbyder endnu en smart pakke: [recipes](https://recipes.tidymodels.org/) - en pakke til databehandling. Vi vil definere en `recipe`, der angiver, at alle forudsigende kolonner skal kodes til et sæt heltal, `prep` den for at estimere de nødvendige mængder og statistikker, der kræves af enhver operation, og til sidst `bake` for at anvende beregningerne på nye data.

> Normalt bruges recipes typisk som en forprocessor til modellering, hvor den definerer, hvilke trin der skal anvendes på et datasæt for at gøre det klar til modellering. I dette tilfælde er det **stærkt anbefalet**, at du bruger en `workflow()` i stedet for manuelt at estimere en recipe ved hjælp af prep og bake. Vi vil se alt dette om lidt.
>
> Men lige nu bruger vi recipes + prep + bake til at angive, hvilke trin der skal anvendes på et datasæt for at gøre det klar til dataanalyse og derefter udtrække de forbehandlede data med de anvendte trin.


In [ ]:
# Preprocess and extract data to allow some data analysis
baked_pumpkins <- recipe(color ~ ., data = pumpkins_select) %>%
  # Define ordering for item_size column
  step_mutate(item_size = ordered(item_size, levels = c('sml', 'med', 'med-lge', 'lge', 'xlge', 'jbo', 'exjbo'))) %>%
  # Convert factors to numbers using the order defined above (Ordinal encoding)
  step_integer(item_size, zero_based = F) %>%
  # Encode all other predictors using one hot encoding
  step_dummy(all_nominal(), -all_outcomes(), one_hot = TRUE) %>%
  prep(data = pumpkin_select) %>%
  bake(new_data = NULL)

# Display the first few rows of preprocessed data
baked_pumpkins %>% 
  slice_head(n = 5)


✅ Hvad er fordelene ved at bruge en ordinal encoder til Item Size-kolonnen?

### Analysér forholdet mellem variabler

Nu hvor vi har forbehandlet vores data, kan vi analysere forholdet mellem funktionerne og etiketten for at få en idé om, hvor godt modellen vil kunne forudsige etiketten baseret på funktionerne. Den bedste måde at udføre denne type analyse på er ved at plotte dataene.  
Vi vil igen bruge ggplot geom_boxplot_-funktionen til at visualisere forholdet mellem Item Size, Variety og Color i et kategorisk plot. For at plotte dataene bedre vil vi bruge den kodede Item Size-kolonne og den ukodede Variety-kolonne.


In [ ]:
# Define the color palette
palette <- c(ORANGE = "orange", WHITE = "wheat")

# We need the encoded Item Size column to use it as the x-axis values in the plot
pumpkins_select_plot<-pumpkins_select
pumpkins_select_plot$item_size <- baked_pumpkins$item_size

# Create the grouped box plot
ggplot(pumpkins_select_plot, aes(x = `item_size`, y = color, fill = color)) +
  geom_boxplot() +
  facet_grid(variety ~ ., scales = "free_x") +
  scale_fill_manual(values = palette) +
  labs(x = "Item Size", y = "") +
  theme_minimal() +
  theme(strip.text = element_text(size = 12)) +
  theme(axis.text.x = element_text(size = 10)) +
  theme(axis.title.x = element_text(size = 12)) +
  theme(axis.title.y = element_blank()) +
  theme(legend.position = "bottom") +
  guides(fill = guide_legend(title = "Color")) +
  theme(panel.spacing = unit(0.5, "lines"))+
  theme(strip.text.y = element_text(size = 4, hjust = 0)) 


#### Brug et swarm-plot

Da Farve er en binær kategori (Hvid eller Ikke-hvid), kræver det 'en [specialiseret tilgang](https://github.com/rstudio/cheatsheets/blob/main/data-visualization.pdf) til visualisering'.

Prøv et `swarm-plot` for at vise fordelingen af farve i forhold til item_size.

Vi vil bruge [ggbeeswarm-pakken](https://github.com/eclarke/ggbeeswarm), som tilbyder metoder til at skabe beeswarm-stil plots ved hjælp af ggplot2. Beeswarm-plots er en måde at plotte punkter, der normalt ville overlappe, så de i stedet placeres ved siden af hinanden.


In [ ]:
# Create beeswarm plots of color and item_size
baked_pumpkins %>% 
  mutate(color = factor(color)) %>% 
  ggplot(mapping = aes(x = color, y = item_size, color = color)) +
  geom_quasirandom() +
  scale_color_brewer(palette = "Dark2", direction = -1) +
  theme(legend.position = "none")


Nu hvor vi har en idé om forholdet mellem de binære farvekategorier og den større gruppe af størrelser, lad os undersøge logistisk regression for at bestemme en given græskars sandsynlige farve.

## Byg din model

Vælg de variabler, du vil bruge i din klassifikationsmodel, og opdel dataene i trænings- og testdatasæt. [rsample](https://rsample.tidymodels.org/), en pakke i Tidymodels, tilbyder infrastruktur til effektiv dataopdeling og genprøvning:


In [ ]:
# Split data into 80% for training and 20% for testing
set.seed(2056)
pumpkins_split <- pumpkins_select %>% 
  initial_split(prop = 0.8)

# Extract the data in each split
pumpkins_train <- training(pumpkins_split)
pumpkins_test <- testing(pumpkins_split)

# Print out the first 5 rows of the training set
pumpkins_train %>% 
  slice_head(n = 5)


🙌 Vi er nu klar til at træne en model ved at tilpasse træningsfunktionerne til træningsetiketten (farve).

Vi starter med at oprette en opskrift, der angiver de forbehandlingsskridt, der skal udføres på vores data for at gøre dem klar til modellering, dvs.: kodning af kategoriske variabler til et sæt heltal. Ligesom `baked_pumpkins` opretter vi en `pumpkins_recipe`, men vi `prep` og `bake` ikke, da det vil blive samlet i en arbejdsgang, som du vil se om få trin.

Der er en del måder at specificere en logistisk regressionsmodel i Tidymodels. Se `?logistic_reg()`. For nu vil vi specificere en logistisk regressionsmodel via den standard `stats::glm()` motor.


In [ ]:
# Create a recipe that specifies preprocessing steps for modelling
pumpkins_recipe <- recipe(color ~ ., data = pumpkins_train) %>% 
  step_mutate(item_size = ordered(item_size, levels = c('sml', 'med', 'med-lge', 'lge', 'xlge', 'jbo', 'exjbo'))) %>%
  step_integer(item_size, zero_based = F) %>%  
  step_dummy(all_nominal(), -all_outcomes(), one_hot = TRUE)

# Create a logistic model specification
log_reg <- logistic_reg() %>% 
  set_engine("glm") %>% 
  set_mode("classification")


Nu hvor vi har en opskrift og en modelspecifikation, skal vi finde en måde at samle dem i et objekt, der først forbehandler dataene (prep+bake bag kulisserne), tilpasser modellen på de forbehandlede data og også giver mulighed for potentielle efterbehandlingsaktiviteter.

I Tidymodels kaldes dette praktiske objekt en [`workflow`](https://workflows.tidymodels.org/) og holder bekvemt dine modelkomponenter.


In [ ]:
# Bundle modelling components in a workflow
log_reg_wf <- workflow() %>% 
  add_recipe(pumpkins_recipe) %>% 
  add_model(log_reg)

# Print out the workflow
log_reg_wf


Efter en arbejdsgang er blevet *specificeret*, kan en model `trænes` ved hjælp af [`fit()`](https://tidymodels.github.io/parsnip/reference/fit.html)-funktionen. Arbejdsgangen vil estimere en opskrift og forbehandle dataene før træning, så vi ikke behøver manuelt at gøre det ved hjælp af prep og bake.


In [ ]:
# Train the model
wf_fit <- log_reg_wf %>% 
  fit(data = pumpkins_train)

# Print the trained workflow
wf_fit


Modellen viser de koefficienter, der er lært under træningen.

Nu hvor vi har trænet modellen med træningsdataene, kan vi lave forudsigelser på testdataene ved hjælp af [parsnip::predict()](https://parsnip.tidymodels.org/reference/predict.model_fit.html). Lad os starte med at bruge modellen til at forudsige etiketter for vores testdatasæt og sandsynlighederne for hver etiket. Når sandsynligheden er mere end 0.5, er den forudsagte klasse `WHITE`, ellers `ORANGE`.


In [ ]:
# Make predictions for color and corresponding probabilities
results <- pumpkins_test %>% select(color) %>% 
  bind_cols(wf_fit %>% 
              predict(new_data = pumpkins_test)) %>%
  bind_cols(wf_fit %>%
              predict(new_data = pumpkins_test, type = "prob"))

# Compare predictions
results %>% 
  slice_head(n = 10)


Meget flot! Dette giver nogle flere indsigter i, hvordan logistisk regression fungerer.

### Bedre forståelse via en forvirringsmatrix

At sammenligne hver forudsigelse med dens tilsvarende "ground truth" faktiske værdi er ikke en særlig effektiv måde at afgøre, hvor godt modellen forudsiger. Heldigvis har Tidymodels nogle flere tricks i ærmet: [`yardstick`](https://yardstick.tidymodels.org/) - en pakke, der bruges til at måle effektiviteten af modeller ved hjælp af præstationsmålinger.

En præstationsmåling, der er forbundet med klassifikationsproblemer, er [`forvirringsmatrix`](https://wikipedia.org/wiki/Confusion_matrix). En forvirringsmatrix beskriver, hvor godt en klassifikationsmodel præsterer. En forvirringsmatrix opstiller, hvor mange eksempler i hver klasse der blev korrekt klassificeret af en model. I vores tilfælde vil den vise dig, hvor mange orange græskar der blev klassificeret som orange, og hvor mange hvide græskar der blev klassificeret som hvide; forvirringsmatrixen viser også, hvor mange der blev klassificeret i de **forkerte** kategorier.

Funktionen [**`conf_mat()`**](https://tidymodels.github.io/yardstick/reference/conf_mat.html) fra yardstick beregner denne krydstabulering af observerede og forudsagte klasser.


In [ ]:
# Confusion matrix for prediction results
conf_mat(data = results, truth = color, estimate = .pred_class)


Lad os fortolke forvirringsmatricen. Vores model skal klassificere græskar mellem to binære kategorier, kategorien `hvid` og kategorien `ikke-hvid`.

-   Hvis din model forudsiger et græskar som hvidt, og det faktisk tilhører kategorien 'hvid', kalder vi det en `true positive`, vist ved tallet øverst til venstre.

-   Hvis din model forudsiger et græskar som ikke-hvidt, og det faktisk tilhører kategorien 'hvid', kalder vi det en `false negative`, vist ved tallet nederst til venstre.

-   Hvis din model forudsiger et græskar som hvidt, og det faktisk tilhører kategorien 'ikke-hvid', kalder vi det en `false positive`, vist ved tallet øverst til højre.

-   Hvis din model forudsiger et græskar som ikke-hvidt, og det faktisk tilhører kategorien 'ikke-hvid', kalder vi det en `true negative`, vist ved tallet nederst til højre.

| Sandhed |
|:--------:|

|               |        |       |
|---------------|--------|-------|
| **Forudsagt** | HVID   | ORANGE |
| HVID          | TP     | FP    |
| ORANGE        | FN     | TN    |

Som du måske har gættet, er det at foretrække at have et større antal true positives og true negatives og et lavere antal false positives og false negatives, hvilket indikerer, at modellen præsterer bedre.

Forvirringsmatricen er nyttig, da den giver anledning til andre metrikker, der kan hjælpe os med bedre at evaluere ydeevnen af en klassifikationsmodel. Lad os gennemgå nogle af dem:

🎓 Præcision: `TP/(TP + FP)` defineret som andelen af forudsagte positive, der faktisk er positive. Også kaldet [positiv prædiktiv værdi](https://en.wikipedia.org/wiki/Positive_predictive_value "Positive predictive value").

🎓 Recall: `TP/(TP + FN)` defineret som andelen af positive resultater ud af antallet af prøver, der faktisk var positive. Også kendt som `sensitivitet`.

🎓 Specificitet: `TN/(TN + FP)` defineret som andelen af negative resultater ud af antallet af prøver, der faktisk var negative.

🎓 Nøjagtighed: `TP + TN/(TP + TN + FP + FN)` Procentdelen af etiketter, der er korrekt forudsagt for en prøve.

🎓 F-mål: Et vægtet gennemsnit af præcision og recall, hvor det bedste er 1 og det værste er 0.

Lad os beregne disse metrikker!


In [ ]:
# Combine metric functions and calculate them all at once
eval_metrics <- metric_set(ppv, recall, spec, f_meas, accuracy)
eval_metrics(data = results, truth = color, estimate = .pred_class)


## Visualiser ROC-kurven for denne model

Lad os lave endnu en visualisering for at se den såkaldte [`ROC-kurve`](https://en.wikipedia.org/wiki/Receiver_operating_characteristic):


In [ ]:
# Make a roc_curve
results %>% 
  roc_curve(color, .pred_ORANGE) %>% 
  autoplot()


ROC-kurver bruges ofte til at få et overblik over en klassifikators output i forhold til dens sande vs. falske positiver. ROC-kurver viser typisk `True Positive Rate`/Sensitivitet på Y-aksen og `False Positive Rate`/1-Specificitet på X-aksen. Derfor er kurvens hældning og afstanden mellem midterlinjen og kurven vigtige: du ønsker en kurve, der hurtigt bevæger sig opad og væk fra linjen. I vores tilfælde er der falske positiver i starten, og derefter bevæger linjen sig korrekt opad og væk.

Til sidst kan vi bruge `yardstick::roc_auc()` til at beregne det faktiske areal under kurven (Area Under the Curve). En måde at fortolke AUC på er som sandsynligheden for, at modellen rangerer et tilfældigt positivt eksempel højere end et tilfældigt negativt eksempel.


In [ ]:
# Calculate area under curve
results %>% 
  roc_auc(color, .pred_ORANGE)


Resultatet er omkring `0.975`. Da AUC spænder fra 0 til 1, ønsker du en høj score, da en model, der er 100% korrekt i sine forudsigelser, vil have en AUC på 1; i dette tilfælde er modellen *ret god*.

I fremtidige lektioner om klassifikationer vil du lære, hvordan du kan forbedre modellens resultater (såsom at håndtere ubalancerede data i dette tilfælde).

## 🚀Udfordring

Der er meget mere at udforske omkring logistisk regression! Men den bedste måde at lære på er at eksperimentere. Find et datasæt, der egner sig til denne type analyse, og byg en model med det. Hvad lærer du? tip: prøv [Kaggle](https://www.kaggle.com/search?q=logistic+regression+datasets) for interessante datasæt.

## Gennemgang & Selvstudie

Læs de første par sider af [denne artikel fra Stanford](https://web.stanford.edu/~jurafsky/slp3/5.pdf) om nogle praktiske anvendelser af logistisk regression. Tænk over opgaver, der er bedre egnet til den ene eller den anden type regressionsopgaver, som vi har studeret indtil nu. Hvad ville fungere bedst?



---

**Ansvarsfraskrivelse**:  
Dette dokument er blevet oversat ved hjælp af AI-oversættelsestjenesten [Co-op Translator](https://github.com/Azure/co-op-translator). Selvom vi bestræber os på nøjagtighed, skal du være opmærksom på, at automatiserede oversættelser kan indeholde fejl eller unøjagtigheder. Det originale dokument på dets oprindelige sprog bør betragtes som den autoritative kilde. For kritisk information anbefales professionel menneskelig oversættelse. Vi er ikke ansvarlige for eventuelle misforståelser eller fejltolkninger, der opstår som følge af brugen af denne oversættelse.
